In [1]:
import praw
import pandas as pd
import datetime
import time

In [2]:
reddit = praw.Reddit('bot1') # config in ~/.config/praw.ini

In [3]:
def scan_thread(submission):
    # scan a thread to read all submissions/comments
    
    # add the submission itself
    if submission.author is None:
        ds_sub = pd.DataFrame()
    else:
        ds_sub = pd.DataFrame(data = [[submission.id, submission.title, submission.author.name,
                                      submission.author_flair_text,
                                      datetime.datetime.fromtimestamp(submission.created_utc),
                                      submission.selftext]],
                              columns = ['id', 'title', 'author', 'flair', 'created', 'text']);
    
    # now go through the comments
    ds = pd.DataFrame()
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:]  # Seed with top-level
    for comment in comment_queue:
        if comment.author is None:
            continue
        entry = pd.DataFrame(data = [[comment.id, comment.author.name, comment.author_flair_text,
                                     datetime.datetime.fromtimestamp(comment.created_utc),
                                     comment.body, comment.parent_id]],
                             columns = ['id', 'author', 'flair', 'created', 'text', 'parent']);
        ds = ds.append(entry, ignore_index = True)
            
    return ds_sub, ds

In [4]:
subs = pd.read_csv('nba_submissions_reg18.csv', usecols = ['id'])

In [5]:
#ds_sub = pd.DataFrame()
#ds = pd.DataFrame()
#fsub_count = 1
#fcom_count = 1
#sub_count = 1

In [6]:
ds_sub = pd.DataFrame()
ds = pd.DataFrame()
fsub_count = 7
fcom_count = 11
sub_count = 0

In [7]:
skip = True
tstart = time.time()
for sub_id in subs['id']:
    sub_count += 1
    
    if skip:
        if sub_id == 'awrchj':
            skip = False
        else:
            continue

    if sub_count%1000 == 1:
        print(sub_count)
        tcur = time.time()
        print(round((tcur-tstart)/60, 2))
        tstart = tcur
    
    sub = reddit.submission(id=sub_id)
    single_sub, single_ds = scan_thread(sub)
    ds_sub = ds_sub.append(single_sub, ignore_index = True)
    ds = ds.append(single_ds, ignore_index = True)
    if ds.shape[0] > 200000:
        fname = 'nba_comments_reg18_'+ str(fcom_count) + '.csv'
        print(fname)
        ds.to_csv(fname)
        fcom_count += 1
        ds = pd.DataFrame()
    if ds_sub.shape[0] > 200000:
        fname = 'nba_submissions_reg18_' + str(fsub_count) + '.csv'
        print(fname)
        ds_sub.to_csv(fname)
        fsub_count += 1
        ds_sub = pd.DataFrame()

113001
8.65
114001
12.33
115001
16.72
116001
11.05
117001
13.86
118001
11.45
119001
12.44
120001
15.74
121001
17.19
122001
14.32
123001
12.9
124001
11.82
125001
12.02
126001
16.74
nba_comments_reg18_11.csv
127001
15.16
128001
9.51
129001
11.08
130001
11.9
131001
8.64
132001
8.92
133001
15.85
134001
13.7
135001
14.62
136001
13.95
137001
11.54
138001
12.97
139001
14.5


In [8]:
print(fcom_count)
print(fsub_count)

12
7


In [11]:
ds.to_csv('nba_comments_reg18_'+ str(fcom_count) + '.csv')
ds_sub.to_csv('nba_submissions_reg18_' + str(fsub_count) + '.csv')

In [9]:
ds

,id,author,flair,created,text,parent
0,ej5oanp,JokicOrBust,Nuggets,2019-03-22 19:16:22,Wizards getting Zion is the weirdest thing ever,t3_b4bxpb
1,ej5od7a,BIizard,[SAC] Harry Giles,2019-03-22 19:17:15,Hawks. They try hard everygame and they will b...,t3_b4bxpb
2,ej5ozzp,Roger--Smith,Supersonics,2019-03-22 19:25:22,SuperSonics.,t3_b4bxpb
3,ej5pd4z,YaBoiWhit,NBA,2019-03-22 19:30:03,Hawks\n\nTimberwolves\n\nLakers (Watching the ...,t3_b4bxpb
4,ej5pr7h,bolochung3,None,2019-03-22 19:35:08,Charlotte for at least trying these past few y...,t3_b4bxpb
...,...,...,...,...,...,...
167064,ekix5ge,7pharaohs,[GSW] Stephen Curry,2019-04-10 00:09:01,"Hes had a rough go of it, at his age the last ...",t3_bbhkql
167065,ekixh8k,Titronnica,Spurs,2019-04-10 00:14:28,"I'm not crying, you're crying. ;---;",t3_bbhkql
167066,ekix9nt,WickedFierce1,Warriors,2019-04-10 00:10:58,I must be drunk. I read it how it was meant.,t3_bbhkql
167067,ekiwmkp,LebronIsOld,Celtics,2019-04-10 00:00:45,2 years too late imo. He was embarassing out t...,t3_bbhkql


In [10]:
ds_sub

,id,title,author,flair,created,text
0,awrchj,[Josh Hart] Can you explain this foul? @Offici...,grantandreng,Lakers,2019-03-03 01:20:33,
1,awrd78,"Knicks are 2-1 in the last 3 games, while last...",mengobri,Knicks,2019-03-03 01:23:18,[removed]
2,awrdov,The only positive thing for the Lakers fans ri...,Xeanle7,None,2019-03-03 01:25:15,[removed]
3,awrdto,The Los Angeles Lakers (30-33) have been emoti...,hootsboots,Pistons,2019-03-03 01:25:47,[removed]
4,awreuf,Wherewith Blenders Eyewear Creates Worthwhile ...,ebulkemailmarketing,None,2019-03-03 01:29:51,
...,...,...,...,...,...,...
18393,bbhk9z,WATCH THIS GAME AND TELL ME HARDEN IS THE MVP.,nbafanlover,NBA,2019-04-09 23:57:24,[removed]
18394,bbhkau,"Tonight, Dirk became the oldest player in NBA ...",suzukigun4life,[DAL] Wang Zhizhi,2019-04-09 23:57:29,"In what turned out to be his final home game, ..."
18395,bbhkk1,About tonight..,CEA19,Lakers,2019-04-09 23:58:17,[removed]
18396,bbhkk9,What would you rank as the top 5 crossovers of...,DaranBallerz,Magic,2019-04-09 23:58:17,[removed]
